In [3]:
import data_loading
import data_augment
import model_arch

2023-04-25 10:26:13.722548: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 10:26:13.920812: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-25 10:26:13.952911: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-25 10:26:13.952921: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [4]:
ins, outs = data_loading.load_data()

In [5]:
ins.shape

(350, 600, 100, 1)

In [6]:
outs.shape

(350, 600, 100, 1)

In [7]:
from skimage.io import imshow
from matplotlib import pyplot

In [8]:
import numpy as np

In [9]:
in_rows = ins.reshape((ins.shape[0] * ins.shape[1], ins.shape[2], ins.shape[3]))[:,:,0]

In [10]:
in_rows.shape

(210000, 100)

In [11]:
out_rows = outs.reshape((outs.shape[0] * outs.shape[1], outs.shape[2], outs.shape[3]))

In [12]:
out_rows.shape

(210000, 100, 1)

In [13]:
#out_sums = out_rows.sum(axis=(1,2))
present = (out_rows.sum(axis=(1,2)) > 0).astype(np.int32)

In [14]:
#present[present == 1] = 1
present[present == 0] = -1
print(-1 in present)

True


In [15]:
present.shape

(210000,)

In [16]:
from sklearn.svm import SVC

In [17]:
#classifier = SVC(C=0.9)

In [18]:
normalized = (in_rows.transpose() / (np.sqrt((in_rows**2).sum(axis=1)  + 1e-7))).transpose()

In [18]:
normalized.shape

(210000, 100)

In [19]:
classifier.fit(normalized, present)

SVC(C=0.9)

In [1]:
import pickle

In [25]:

with open("presence.svm", "wb") as file:
    pickle.dump(classifier, file)
print("done")

done


In [2]:
with open("presence.svm", "rb") as file:
    class2 = pickle.load(file)
print("loaded")

loaded


In [19]:
predicted_present = class2.predict(normalized)

In [22]:
print(predicted_present.shape)
print(present.shape)
fraction = (predicted_present == present).sum() / present.shape[0]
print(fraction)

(210000,)
(210000,)
0.8239428571428571


In [23]:
from sklearn.svm import SVR
regressor = SVR(C=0.9)
present_rows = normalized[present]
regr_targets = out_rows.argmax(axis=1)[present]

In [29]:
regr_targets = out_rows.argmax(axis=1)[present == 1][:,0]
print(regr_targets.shape)
present_rows = normalized[present == 1]
print(present_rows.shape)

(44084,)
(44084, 100)


In [30]:
regressor.fit(present_rows, regr_targets)

SVR(C=0.9)

done


In [36]:
pred_pos = regressor.predict(present_rows)

In [41]:
with open("position.svm", "wb") as file:
    pickle.dump(regressor, file)
print("done")

done


In [37]:
mean_square_diff = np.mean((pred_pos - regr_targets)**2)
print(mean_square_diff)

223.26835275216783


In [39]:
close_enough = (pred_pos - regr_targets) < 5
print(close_enough.sum() / close_enough.shape[0])

0.7376826059341257


In [40]:
from sklearn.model_selection import train_test_split